# LLDM

A LLM-powered Dungeon Master.

## Requirements

In [1]:
! pdm install

⠋ Resolving packages from lockfile.....
All packages are synced to date, nothing to do.
5l
🎉 All complete!



## Download D&D 5e Resources

In [2]:
! git clone https://github.com/aurorabuilder/elements || echo "Aurora Elements already cloned"
! git clone https://github.com/Tabyltop/CC-SRD || echo "CC-SRD already cloned"
! wget -O srd.zip "https://www.dropbox.com/scl/fi/h6zfhincgxwydijlf8ixk/srd.zip?rlkey=u9yu5r1ly1273st5fu84664ag&dl=0"
! unzip -o srd.zip

fatal: destination path 'elements' already exists and is not an empty directory.
Aurora Elements already cloned
fatal: destination path 'CC-SRD' already exists and is not an empty directory.
CC-SRD already cloned
--2024-01-27 12:10:24--  https://www.dropbox.com/scl/fi/h6zfhincgxwydijlf8ixk/srd.zip?rlkey=u9yu5r1ly1273st5fu84664ag&dl=0
Resolving www.dropbox.com (www.dropbox.com)... 2620:100:6025:18::a27d:4512, 162.125.69.18
Connecting to www.dropbox.com (www.dropbox.com)|2620:100:6025:18::a27d:4512|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7ab4de467e29ae4ce5bf422a62.dl.dropboxusercontent.com/cd/0/inline/CMJSMqyqbglcxI4S3rnH_VwuHM0ZcPtk_XIhQcLPulbBNTafkqywlQCZ8rcoLWVzIFBXplxHbYbJfz99YL_x3NjxmuVg9PgI78RV1XuvIaD0xM6rSrv329GZJudLC4pQJrl4Mn2X8gk3B58I6SoVONeu/file# [following]
--2024-01-27 12:10:24--  https://uc7ab4de467e29ae4ce5bf422a62.dl.dropboxusercontent.com/cd/0/inline/CMJSMqyqbglcxI4S3rnH_VwuHM0ZcPtk_XIhQcLPulbBNTafkqywlQCZ8rcoLWVzIFBXplxHb

## LLM

In [4]:
import torch, torchvision
print(torch.__version__)
torch.cuda.is_available()
print(torchvision.__version__)

2.0.1+cu117
0.15.2+cu117


In [3]:
# add .. to python path
import sys
sys.path.append("..")

from src.models import ModelFactory, ModelType

llm = ModelFactory.get_model(ModelType.LOCAL_API)

## Vector Store

In [4]:
import os
import fnmatch

def find_files_with_extension(root_dir, extension):
    file_list = []
    for root, _, files in os.walk(root_dir):
        for filename in fnmatch.filter(files, f'*.{extension}'):
            file_list.append(os.path.join(root, filename))
    return file_list

# input_files = ["CC-SRD/SRD5.1-CCBY4.0License-TT.txt"]
input_files = find_files_with_extension("srd", "json")
input_files.remove("srd/index.json")

In [5]:
from llama_index import SimpleDirectoryReader
from llama_index import Document

documents = SimpleDirectoryReader(input_files=input_files).load_data()

In [6]:
documents[0].get_text()

'{\n  "count": 24,\n  "results": [\n    {\n      "index": "ability-scores",\n      "name": "Ability Scores",\n      "url": "/api/ability-scores"\n    },\n    {\n      "index": "alignments",\n      "name": "Alignments",\n      "url": "/api/alignments"\n    },\n    {\n      "index": "backgrounds",\n      "name": "Backgrounds",\n      "url": "/api/backgrounds"\n    },\n    {\n      "index": "classes",\n      "name": "Classes",\n      "url": "/api/classes"\n    },\n    {\n      "index": "conditions",\n      "name": "Conditions",\n      "url": "/api/conditions"\n    },\n    {\n      "index": "damage-types",\n      "name": "Damage Types",\n      "url": "/api/damage-types"\n    },\n    {\n      "index": "equipment",\n      "name": "Equipment",\n      "url": "/api/equipment"\n    },\n    {\n      "index": "equipment-categories",\n      "name": "Equipment Categories",\n      "url": "/api/equipment-categories"\n    },\n    {\n      "index": "feats",\n      "name": "Feats",\n      "url": "/api/fe

In [7]:
from llama_index import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.llms.openai import OpenAI
from sentence_transformers import SentenceTransformer

# llm = OpenAI()

# load the embedding model
# embedding_model = SentenceTransformer("BAAI/bge-large-en-v1.5", device="cuda")

auto_merging_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-large-en-v1.5"
)

storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(documents)

automerging_index = VectorStoreIndex(
    documents,
    storage_context=storage_context,
    service_context=auto_merging_context,
    show_progress=True
)

Generating embeddings:   0%|          | 0/2034 [00:00<?, ?it/s]

In [8]:
from llama_index.postprocessor import SentenceEmbeddingOptimizer
from llama_index.indices.postprocessor import SentenceTransformerRerank

rerank = SentenceTransformerRerank(top_n=6, model="BAAI/bge-reranker-base")

query_engine = automerging_index.as_query_engine(
    similarity_top_k=20,
    response_mode="tree_summarize",
    node_postprocessors=[
        rerank,
        # SentenceEmbeddingOptimizer(embed_model="local:BAAI/bge-large-en-v1.5", percentile_cutoff=0.5),
    ]
)

In [9]:
response = query_engine.query(
    "What happens when trying to shoot an enemy hidden behind a railing with a bow?"
)
print(str(response))

/home/diego/source/ll-dm/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


When shooting a hidden enemy behind a railing with a bow, you would face challenges due to the cover provided by the railing. In this situation, the railing is considered as half cover, granting the target a +2 bonus to AC and Dexterity saving throws. As the bow is a ranged weapon, the attack roll would have disadvantage since the target is behind an obstacle. The obstacle would not completely block the shot, but make it more difficult to hit the target. Other factors such as the archer's skill, the railing's range and the target's movement can influence the outcome of the shot.


#### Evaluation with trulens

##### 0. Setup
Setup trulens and patch openai to use the local LLM

In [9]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [26]:
# import openai
# openai.api_key = "1234"
# openai.base_url = "http://localhost:5000/v1"

Allow nested execution, we will need this to run the dashboard AND the notebook

In [10]:
import nest_asyncio

nest_asyncio.apply()

Create the feedback provider, this will be the LLM evaluating our retrieval engine's answers

`Question -> RAG -> LLM to provide an answer -> Another LLM to evaluate the answer`

In [11]:
from trulens_eval import OpenAI as fOpenAI

provider = fOpenAI()

##### 1. Answer relevance

In [12]:
from trulens_eval import Feedback

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


##### 2. Context relevance

In [13]:
from trulens_eval import TruLlama
import numpy as np

context_selection = TruLlama.select_source_nodes().node.text

f_qs_relevance = (
    Feedback(provider.qs_relevance,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


##### 3. Groundedness

In [14]:
from trulens_eval.feedback import Groundedness

grounded = Groundedness(groundedness_provider=provider)

f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons,
             name="Groundedness"
            )
    .on(context_selection)
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


##### Run the evaluation

In [175]:
from trulens_eval import TruLlama
from trulens_eval import FeedbackMode

tru_recorder = TruLlama(
    query_engine,
    app_id="App_3",
    feedbacks=[
        f_qa_relevance,
        f_qs_relevance,
        f_groundedness
    ]
)

In [176]:
# write 10 D&D 5.0 rule questions
questions = [
    "What happens when trying to shoot an enemy hidden behind a railing with a bow?",
    "How do I determine if a creature is within range?",
    "Can I use a bonus action and an action to cast two spells?",
    "Can a bard use a musical instrument as a spellcasting focus?",
    "Can a barbarian cast spells?",
    "What are possible components of a spell?", 
    "How much does a crossbow cost?",
    "How many types of crossbow are there?",
    "Which are the types of crossbow?"  
]

In [177]:
from tqdm import tqdm 
for question in tqdm(questions):
    with tru_recorder as recording:
        query_engine.query(question)

  0%|          | 0/9 [00:00<?, ?it/s]A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7ff1041dfe20 is calling an instrumented method <function BaseQueryEngine.query at 0x7ff410142440>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7ff326b3cac0) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7ff1041dfe20 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7ff32948f7f0>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7ff326b3cac0) using this function.
A new object of type <class 'llama_index.indices.vector_store.retrievers.retriever.VectorIndexRetriever'> at 0x7ff0cbc87cd0 is calling an instrumented method <function BaseRetriever.retrieve at 0x7ff3dba05e10>. The path of this call may be incorrect.
Guessing path of new objec

##### Dashboard
The cell below launches a streamlit dashboard that shows the evaluation details

In [178]:
tru.get_leaderboard(app_ids=[])
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.1.172:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Agents

Set the debug options to be able to follow the chain's execution

In [37]:
from langchain.globals import set_debug, set_verbose

use_the_thanos_gauntlet_of_logs = True
set_debug(use_the_thanos_gauntlet_of_logs)
set_verbose(use_the_thanos_gauntlet_of_logs)

Make the LLaMaIndex query engine a tool to be used by the LangChain agent

In [19]:
from llama_index.langchain_helpers.agents import (
    IndexToolConfig,
    LlamaIndexTool,
)

tool_config = IndexToolConfig(
    query_engine=query_engine,
    name='rules-index',
    description=f"Useful for when you want to answer queries about D&D rules, mechanics and detailed information such as costs, ranges, etc. Input must be a single string containing a single natural language question",
    tool_kwargs={"return_direct": False},
)

tool = LlamaIndexTool.from_tool_config(tool_config)

In [189]:
from langchain import hub
from langchain.prompts import SystemMessagePromptTemplate, PromptTemplate

template = """You are an agent managing the rules of D&D. You are an expert of the D&D game and you have access to the D&D rules through an index tool. \
You must answer to questions related to the D&D rules by solely relying on the information retrieved with the provided tool. Do NOT rely on prior knowledge. \
Make sure to check all the relevant resources related to the question using the D&D rules index tool provided. \
If you are not able to find the requested information, answer that you were not able to find the requested information and ask for clarifications."""

pt = PromptTemplate(input_variables=[], template=template)
smpt = SystemMessagePromptTemplate(prompt=pt)
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages[0] = smpt
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an agent managing the rules of D&D. You are an expert of the D&D game and you have access to the D&D rules through an index tool. You must answer to questions related to the D&D rules by solely relying on the information retrieved with the provided tool. Do NOT rely on prior knowledge. Make sure to check all the relevant resources related to the question using the D&D rules index tool provided. If you are not able to find the requested information, answer that you were not able to find the requested information and ask for clarifications.')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [190]:
# react_prompt = hub.pull("hwchase17/react")
react_prompt = hub.pull("hwchase17/react-json")
print(react_prompt.messages[0])


prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the "action" field are: {tool_names}\n\nThe $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nALWAYS use the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction:\n```\n$JSON_BLOB\n```\nObservation: the result of the action\n... (this Thought/Action/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer

In [201]:
prompt = hub.pull("hwchase17/react-json")
print(type(prompt))
for message in prompt.messages:
    print(message.prompt.template)
    print("--------------------------")

<class 'langchain_core.prompts.chat.ChatPromptTemplate'>
Answer the following questions as best you can. You have access to the following tools:

{tools}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are: {tool_names}

The $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action:
```
$JSON_BLOB
```
Observation: the result of the action
... (this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remin

In [207]:
prompt.messages[0].prompt = prompt.messages[0].prompt.from_template("""Answer the following questions as best you can. You have access to the following tools:

{tools}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are: {tool_names}

The $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action:
```
$JSON_BLOB
... (The Thought and Action sections can be repeated until the question is answered)
{input}

{agent_scratchpad}
""")

print(prompt.messages[0].prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are: {tool_names}

The $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action:
```
$JSON_BLOB
... (The Thought and Action sections can be repeated until the question is answered)
{input}

{agent_scratchpad}



In [24]:
from langchain import hub
from langchain.agents import AgentExecutor, load_tools, create_structured_chat_agent
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)
from langchain.tools.render import render_text_description

import langchain
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()

# setup tools
# tools = load_tools(["serpapi", "llm-math"], llm=llm)
tools = [tool]

# # setup ReAct style prompt
# # prompt = hub.pull("hwchase17/react-json")
# prompt = prompt.partial(
#     tools=render_text_description(tools),
#     tool_names=", ".join([t.name for t in tools]),
# )

# # define the agent
chat_model_with_stop = llm.bind(stop=["\nObservation"])
# agent = (
#     {
#         "input": lambda x: x["input"],
#         "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
#     }
#     | prompt
#     | chat_model_with_stop
#     | ReActJsonSingleInputOutputParser()
# )
# print(prompt.messages[0].prompt.template)

prompt = hub.pull("hwchase17/structured-chat-agent")

# Construct the JSON agent
agent = create_structured_chat_agent(chat_model_with_stop, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [12]:
os.environ["LANGCHAIN_TRACING"] = "true"

In [25]:

agent_executor.invoke(
    {
        "input": "Which are the possible lawful alignment? Give a one sentence description of each.",
    }
)

Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f62ba4cc2e0>: Failed to establish a new connection: [Errno 111] Connection refused'))




> Entering new AgentExecutor chain...


Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f62ba536a70>: Failed to establish a new connection: [Errno 111] Connection refused'))


{
  "action": "rules-index",
  "action_input": "Describe lawful alignments in a single sentence"
}


Error in LangChainTracerV1.on_llm_end callback: ValidationError(model='LLMRun', errors=[{'loc': ('response', 'generations', 0, 0, 'type'), 'msg': "unexpected value; permitted: 'Generation'", 'type': 'value_error.const', 'ctx': {'given': 'ChatGeneration', 'permitted': ('Generation',)}}])


Lawful alignments prioritize order, tradition, or loyalty, as seen in creatures like gold dragons and paladins, who follow a strict moral code.

Error in LangChainTracerV1.on_chain_end callback: ValueError('Unknown run type: prompt')


{
  "action": "Final Answer",
  "action_input": "Lawful alignments in D&D are rooted in obedience, order, and strict moral codes, with examples like gold dragons and paladins."
}

> Finished chain.


{'input': 'Which are the possible lawful alignment? Give a one sentence description of each.',
 'output': 'Lawful alignments in D&D are rooted in obedience, order, and strict moral codes, with examples like gold dragons and paladins.'}

In [26]:
resp = agent_executor.invoke({"input": "Which of the available races can have a Constitution stat bonus?"})
print(resp['output'])

Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f62ba4cff70>: Failed to establish a new connection: [Errno 111] Connection refused'))




> Entering new AgentExecutor chain...
{
  "action": "rules-index",
  "action_input": "Which races have a bonus to Constitution stat in D&D?"
}


Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f62ba594eb0>: Failed to establish a new connection: [Errno 111] Connection refused'))
Error in LangChainTracerV1.on_llm_end callback: ValidationError(model='LLMRun', errors=[{'loc': ('response', 'generations', 0, 0, 'type'), 'msg': "unexpected value; permitted: 'Generation'", 'type': 'value_error.const', 'ctx': {'given': 'ChatGeneration', 'permitted': ('Generation',)}}])
Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f62ba326e00>: Failed to establish a new connection: [Errno 111] Connection refused'))
Error in LangChainTracerV1.on_llm_end callback: ValidationError(model=

Identifying races with a bonus to Constitution stat in D&D involves looking into individual race abilities, as some races might have racial bonuses for specific ability scores, including Constitution. It's essential to consult official D&D sources to find out the specific cases where certain races gain a bonus to Constitution.

Error in LangChainTracerV1.on_chain_end callback: ValueError('Unknown run type: prompt')


{
  "action": "Final Answer",
  "action_input": "Races with Constitution bonuses include Dwarves, Half-Orcs, and Tieflings. Additionally, some subraces have this bonus, such as Mountain Dwarves, Mark of the Wild Half-Orcs, and Striking Tieflings. To be sure, always refer to your D&D source material."
}

> Finished chain.
Races with Constitution bonuses include Dwarves, Half-Orcs, and Tieflings. Additionally, some subraces have this bonus, such as Mountain Dwarves, Mark of the Wild Half-Orcs, and Striking Tieflings. To be sure, always refer to your D&D source material.


In [27]:
resp = agent_executor.invoke({"input": "what's the cost of a crossbow?"})
print(resp['output'])

Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f62a06cd7b0>: Failed to establish a new connection: [Errno 111] Connection refused'))




> Entering new AgentExecutor chain...
Could not parse LLM output: {
  "action": rules-index,
  "action_input": "Input a question about the cost of a crossbow."
}

Invalid or incomplete responseCould not parse LLM output: {
  "action": rules-index,
  "action_input": "Input a question about the cost of a crossbow."
}

Invalid or incomplete responseCould not parse LLM output: {
  "action": rules-index,
  "action_input": "What is the cost of a crossbow?"
}

Invalid or incomplete response

Error in LangChainTracerV1.on_chain_end callback: ValueError('Unknown run type: prompt')


{
  "action": "Final Answer",
  "action_input": "The cost of a crossbow varies depending on factors like its quality, material, and manufacturer. Generally, you can find crossbows in the range of $200 to $700."
}

> Finished chain.
The cost of a crossbow varies depending on factors like its quality, material, and manufacturer. Generally, you can find crossbows in the range of $200 to $700.


In [23]:
resp = agent_executor.invoke({"input": "How many types of crossbows are there? How much do they cost?"})
print(resp['output'])

Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f62ba4cf280>: Failed to establish a new connection: [Errno 111] Connection refused'))




> Entering new AgentExecutor chain...
Could not parse LLM output: Action:
```
{
  "action": rules-index,
  "action_input": "How many types of crossbows are there and what are their costs?"
}
```
Type: Crossbows have multiple types, such as recurve, compound, and repeating crossbows. Costs vary by type, manufacturer, and quality.
Invalid or incomplete response

Error in LangChainTracerV1.on_chain_end callback: ValueError('Unknown run type: prompt')


{
  "action": "Final Answer",
  "action_input": "Crossbows have different types like recurve, compound, and repeating crossbows, and their costs depend on factors such as type, manufacturer, and quality."
}



> Finished chain.
Crossbows have different types like recurve, compound, and repeating crossbows, and their costs depend on factors such as type, manufacturer, and quality.


In [170]:
resp = agent_executor.invoke({"input": "Can a LVL3 barbarian cast Magic Missile?"})
print(resp['output'])

Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7ff0cbe8b3a0>: Failed to establish a new connection: [Errno 111] Connection refused'))




> Entering new AgentExecutor chain...
Question: Can a LVL3 barbarian cast Magic Missile?
Thought: Knowing the class limitations and magic abilities
Action:
```
{
  "action": "rules-index",
  "action_input": "Can a level 3 barbarian cast Magic Missile?"
}
```

Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7ff0cbe14cd0>: Failed to establish a new connection: [Errno 111] Connection refused'))
Error in LangChainTracerV1.on_llm_end callback: ValidationError(model='LLMRun', errors=[{'loc': ('response', 'generations', 0, 0, 'type'), 'msg': "unexpected value; permitted: 'Generation'", 'type': 'value_error.const', 'ctx': {'given': 'ChatGeneration', 'permitted': ('Generation',)}}])


No, a level 3 barbarian cannot cast Magic Missile since they are not a spellcaster class. Magic Missile is a spell, not a class feature, and the barbarian class is focused more on physical combat and rage powers.

Error in LangChainTracerV1.on_chain_end callback: ValueError('Unknown run type: prompt')


Final Answer: No, a level 3 barbarian cannot cast Magic Missile.

> Finished chain.
No, a level 3 barbarian cannot cast Magic Missile.
